# 企业AIGC商业落地应用研究报告分析

## 加载报告

In [2]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("data/report.pdf") # 57 pages pdf document

## 使用 VectorstoreIndexCreator

In [13]:
from langchain.chat_models import ChatOpenAI
from langchain.indexes import VectorstoreIndexCreator
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

result=[]

index = VectorstoreIndexCreator(
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0),
    embedding = OpenAIEmbeddings(),
    vectorstore_cls=Chroma,
    vectorstore_kwargs={"k":2} # defined k as 2 meaning that we are only interested in getting two relevant text chunks.
).from_loaders([loader])
query = "AIGC可以应用在企业里哪些场景？"
index.query(llm=ChatOpenAI(temperature=0.0), question=query, chain_type="stuff", verbose=True)



> Entering new RetrievalQA chain...

> Finished chain.


'根据提供的上下文，AIGC可以应用在十三个热门场景中，其中包括营销场景。在营销场景中，AIGC可以支持性能分析和报告，帮助企业优化营销过程。此外，上下文还提到AIGC在部分场景能够起到作用，但目前仅限于极少部分零星的应用，因此具体应用场景需要根据企业的具体情况来确定。'

## 使用 RetrievalQA

In [16]:
from langchain.chains import RetrievalQA

# split the documents into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(loader.load())
# select which embeddings we want to use
embeddings = OpenAIEmbeddings()
# create the vectorestore to use as the index
db = Chroma.from_documents(texts, embeddings)
# expose this index in a retriever interface
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k":2}) # defined k as 2 meaning that we are only interested in getting two relevant text chunks.
# create a chain to answer questions 
qa = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(temperature=0.0), chain_type="stuff", retriever=retriever, return_source_documents=True, verbose=True)
query = "AIGC可以应用在企业里哪些场景？"
result = qa({"query": query})



> Entering new RetrievalQA chain...

> Finished chain.


In [17]:
result

{'query': 'AIGC可以应用在企业里哪些场景？',
 'result': '根据提供的上下文，AIGC可以应用在企业的营销场景中，支持性能分析和报告，以及小范围业务与应用程序实施的具体业务动作。具体来说，AIGC可以在营销过程中支持性能分析和报告，帮助企业了解营销活动的效果和质量。此外，AIGC还可以在小范围尝试中支持业务与应用程序实施，以帮助企业更好地实现业务目标。',
 'source_documents': [Document(page_content='所以，企业信息化建设掌舵者接受 AIGC的潜在业务影响能力并做出举措\n能够深度影响大量企业场\n景的作业模式在部场景能够起到作用目前还仅限于极少部分零\n星的应用暂时难以判断对企业服务\n的影响在后端应用程序实施部分能力渗透在前端应用程序实施部分能力渗透质量与效果保障和现场测试需求定义具体业务内容支持 ，例如营销过程\n中支持性能分析和报告小范围业务与应用程序实施已经尝试的具体业务动作\n小范围尝试\n性能分析需求定义', metadata={'source': 'data/report.pdf', 'page': 17}),
  Document(page_content='应用场景示例：营销场景 AIGC 应用企业概览\nsource：TE企服指数，数据截至 2023年5月 更多详细数据请关注： https://aigc.iyiou.com/aigc/companylist\nWPS智能写作\n酷家乐\nAlivia\n有连云\n视觉中国加马平台\nAzero -\n壹元系统\n竹间智能\n营销场景\n AIGC\n 应用企业星锐象限\n 营销场景\n AIGC\n 应用企业榜单', metadata={'source': 'data/report.pdf', 'page': 36})]}